```json
  {  "Name": "Portland, OR",

  "Lat": "45.59578°N",

  "Lon": "122.60917°W",

  "Elev": "20ft.",

  "Temperature": "45\u00b0F",

  "Humidity": "75%",

  "Wind Speed": "WSW 8 mph",

  "Barometer": "29.99 in (1015.58 mb)",

  "Dewpoint": "37\u00b0F (3\u00b0C)",

  "Visibility": "10.00 mi",

  "Wind Chill": "41\u00b0F (5\u00b0C)",

  "Last update": "13 Feb 10:15 AM PST"}



  {  "Name": "San Diego, CA",

  "Lat": "32.73361°N",

  "Lon": "117.18306°W",

  "Elev": "13ft.",

  "Temperature": "55\u00b0F",

  "Humidity": "66%",

  "Wind Speed": "SE 3 MPH",

  "Barometer": "30.0 in (1015.92 mb)",

  "Dewpoint": "44\u00b0F (7\u00b0C)",

  "Visibility": "10.00 mi",

  "Wind Chill": "Null",

  "Last update": "13 Feb 09:35 AM PST"}



  {  "Name": "Duluth, MN",

  "Temperature": "39\u00b0F",

  "Humidity": "56%",

  "Wind Speed": "SW 10 mph",

  "Barometer": "27.71 in (1029.8 mb)",

  "Dewpoint": "25\u00b0F (-4\u00b0C)",

  "Visibility": "10.00 mi",

  "Wind Chill": "32\u00b0F (0\u00b0C)",

  "Last update": "13 Feb 12:55 pm CST"}
```

In [32]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

with open('cities.json') as f:
    cities = json.load(f)

data = []

for city in cities:
    response = requests.get(city['NWS_URL'])
    soup = BeautifulSoup(response.content, 'html.parser')

    location = soup.find('h2', {'class': 'panel-title'})
    lat_lon_elev = soup.find('span', {'class': 'smallTxt'}).text.strip()
    lat, lon, elev = re.findall(r'[-+]?\d*\.\d+|\d+', lat_lon_elev)
    temperature = soup.find('p', {'class': 'myforecast-current-lrg'})
    humidity_elem = soup.find('td', text='Humidity')
    humidity = humidity_elem.find_next('td').text.strip() if humidity_elem else 'NA'
    wind_speed_elem = soup.find('td', text='Wind Speed')
    wind_speed = wind_speed_elem.find_next('td').text.strip() if wind_speed_elem else 'NA'
    barometer_elem = soup.find('td', text='Barometer')
    barometer = barometer_elem.find_next('td').text.strip() if barometer_elem else 'NA'
    dewpoint_elem = soup.find('td', text='Dewpoint')
    dewpoint = dewpoint_elem.find_next('td').text.strip() if dewpoint_elem else 'NA'
    visibility_elem = soup.find('td', text='Visibility')
    visibility = visibility_elem.find_next('td').text.strip() if visibility_elem else 'NA'
    wind_chill_elem = soup.find('td', text='Wind Chill')
    wind_chill = wind_chill_elem.find_next('td').text.strip() if wind_chill_elem else 'NA'
    last_update_elem = soup.find('td', text='Last update')
    last_update = last_update_elem.find_next('td').text.strip() if last_update_elem else 'NA'

    data.append({
        'location': city['Name'],
        'lat': lat,
        'lon': lon,
        'elev_ft': elev,
        'temperature': temperature.text if temperature else 'NA',
        'humidity': humidity,
        'wind_speed': wind_speed,
        'barometer': barometer,
        'dewpoint': dewpoint,
        'vis_miles': visibility,
        'wind_chill': wind_chill,
        'last_update': last_update
    })

df = pd.DataFrame(data)
print(df)


/Users/drewwhite/Desktop/Epicodus/team-week3/venv/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
/Users/drewwhite/Desktop/Epicodus/team-week3/venv/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
/Users/drewwhite/Desktop/Epicodus/team-week3/venv/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
/Users/drewwhite/Desktop/Epicodus/team-week3/venv/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
/Users/drewwhite/Desktop/Epicodus/team-week3/venv/lib/python3.7/site-packages/ipykernel_launcher.py:27: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Us

        location       lat        lon elev_ft temperature humidity  \
0   Portland, OR  45.59578  122.60917      20        36°F      86%   
1  San Diego, CA  32.73361  117.18306      13        57°F      67%   
2     Duluth, MN     46.72      92.04     607        34°F      93%   

   wind_speed              barometer    dewpoint vis_miles wind_chill  \
0     N 0 MPH  30.05 in (1017.61 mb)  32°F (0°C)  10.00 mi         NA   
1  WSW 16 MPH   30.0 in (1015.92 mb)  46°F (8°C)  10.00 mi         NA   
2        Calm               29.48 in  32°F (0°C)   8.00 mi         NA   

           last_update  
0  14 Feb 08:20 AM PST  
1  14 Feb 08:20 AM PST  
2  14 Feb 10:55 am CST  


In [29]:
df.dtypes

Location       object
Lat            object
Lon            object
Elev           object
Temperature    object
Humidity       object
Wind Speed     object
Barometer      object
Dewpoint       object
Visibility     object
Wind Chill     object
Last update    object
dtype: object

In [34]:
# Split the 'location' column into separate 'city' and 'state' columns
df[['city', 'state']] = df['location'].str.split(', ', expand=True)

print(df)

        location       lat        lon elev_ft temperature humidity  \
0   Portland, OR  45.59578  122.60917      20        36°F      86%   
1  San Diego, CA  32.73361  117.18306      13        57°F      67%   
2     Duluth, MN     46.72      92.04     607        34°F      93%   

   wind_speed              barometer    dewpoint vis_miles wind_chill  \
0     N 0 MPH  30.05 in (1017.61 mb)  32°F (0°C)  10.00 mi         NA   
1  WSW 16 MPH   30.0 in (1015.92 mb)  46°F (8°C)  10.00 mi         NA   
2        Calm               29.48 in  32°F (0°C)   8.00 mi         NA   

           last_update       city state  
0  14 Feb 08:20 AM PST   Portland    OR  
1  14 Feb 08:20 AM PST  San Diego    CA  
2  14 Feb 10:55 am CST     Duluth    MN  


In [39]:
df[['lat', 'lon']] = df[['lat', 'lon']].astype(float)
df['elev_ft'] = df['elev_ft'].astype(int)
print(df)

        location       lat        lon  elev_ft temperature humidity  \
0   Portland, OR  45.59578  122.60917       20        36°F      86%   
1  San Diego, CA  32.73361  117.18306       13        57°F      67%   
2     Duluth, MN  46.72000   92.04000      607        34°F      93%   

   wind_speed              barometer    dewpoint vis_miles wind_chill  \
0     N 0 MPH  30.05 in (1017.61 mb)  32°F (0°C)  10.00 mi         NA   
1  WSW 16 MPH   30.0 in (1015.92 mb)  46°F (8°C)  10.00 mi         NA   
2        Calm               29.48 in  32°F (0°C)   8.00 mi         NA   

           last_update       city state  
0  14 Feb 08:20 AM PST   Portland    OR  
1  14 Feb 08:20 AM PST  San Diego    CA  
2  14 Feb 10:55 am CST     Duluth    MN  


In [45]:
# Extract the numeric part of the temperature string and convert it to int
df['temp_f'] = df['temperature'].str.extract('(\d+)').astype(int)

# Convert temperature to Celsius and add to new column 'temp_c'
df['temp_c'] = (df['temp_f'] - 32) * 5/9

# Round 'temp_c' to nearest integer and cast to int type
df['temp_c'] = df['temp_c'].round().astype(int)
print(df)

        location       lat        lon  elev_ft temperature humidity  \
0   Portland, OR  45.59578  122.60917       20        36°F      86%   
1  San Diego, CA  32.73361  117.18306       13        57°F      67%   
2     Duluth, MN  46.72000   92.04000      607        34°F      93%   

   wind_speed              barometer    dewpoint vis_miles wind_chill  \
0     N 0 MPH  30.05 in (1017.61 mb)  32°F (0°C)  10.00 mi         NA   
1  WSW 16 MPH   30.0 in (1015.92 mb)  46°F (8°C)  10.00 mi         NA   
2        Calm               29.48 in  32°F (0°C)   8.00 mi         NA   

           last_update       city state  temp_f  temp_c  
0  14 Feb 08:20 AM PST   Portland    OR      36       2  
1  14 Feb 08:20 AM PST  San Diego    CA      57      14  
2  14 Feb 10:55 am CST     Duluth    MN      34       1  
